In [1]:
import julia 
from julia import Julia
jl = Julia(compiled_modules=False)  # Initialize Julia interpreter
Julia(runtime="/home/jxxiong/julia-1.9.2/bin/julia")
from julia import Base
# Now you can call functions from Julia's Base module
from julia import Main

num_samples = 10000

In [2]:
import numpy as np
import gurobipy as gp

from gurobipy import Model, GRB, QuadExpr, LinExpr
from scipy.sparse import csc_matrix
import pickle

In [3]:
Main.eval("""
using PowerModels
using JuMP
using Ipopt

file_path = "/home/jxxiong/A-xjx/DC3/datasets/dcopf/";
case_name = "pglib_opf_case200_activ.m";
network_data_orig = PowerModels.parse_file(file_path*case_name);
optimizer = optimizer_with_attributes(Ipopt.Optimizer, "print_level"=>0);
""")

[warn | PowerModels]: no active generators found at bus 168, updating to bus type from 2 to 1
[warn | PowerModels]: no active generators found at bus 197, updating to bus type from 2 to 1
[warn | PowerModels]: no active generators found at bus 92, updating to bus type from 2 to 1
[warn | PowerModels]: no active generators found at bus 161, updating to bus type from 2 to 1
[warn | PowerModels]: no active generators found at bus 165, updating to bus type from 2 to 1
[warn | PowerModels]: no active generators found at bus 196, updating to bus type from 2 to 1
[warn | PowerModels]: no active generators found at bus 164, updating to bus type from 2 to 1
[warn | PowerModels]: no active generators found at bus 78, updating to bus type from 2 to 1
[warn | PowerModels]: no active generators found at bus 166, updating to bus type from 2 to 1
[warn | PowerModels]: no active generators found at bus 169, updating to bus type from 2 to 1
[warn | PowerModels]: no active generators found at bus 79, up

<PyCall.jlwrap MathOptInterface.OptimizerWithAttributes(Ipopt.Optimizer, Pair{MathOptInterface.AbstractOptimizerAttribute, Any}[MathOptInterface.RawOptimizerAttribute("print_level") => 0])>

In [4]:
def get_constraints(m):
    ini_ineq_matrix = []
    ini_ineq_rhs = []
    ini_eq_matrix = []
    ini_eq_rhs = []
    a_matrix = m.getA().toarray()

    i = 0
    for constr in m.getConstrs():
        if constr.sense == '<':
            ini_ineq_matrix.append(a_matrix[i,:])
            ini_ineq_rhs.append(constr.RHS)
            i += 1
        elif constr.sense == '>':
            ini_ineq_matrix.append(-a_matrix[i,:])
            ini_ineq_rhs.append(-constr.RHS)
            i += 1
        elif constr.sense == '=':
            ini_eq_matrix.append(a_matrix[i,:])
            ini_eq_rhs.append(constr.RHS)
            i += 1

    ini_ineq_matrix = np.array(ini_ineq_matrix)
    ini_ineq_rhs = np.array(ini_ineq_rhs)
    ini_eq_matrix = np.array(ini_eq_matrix)
    ini_eq_rhs = np.array(ini_eq_rhs)

    return ini_ineq_matrix, ini_ineq_rhs, ini_eq_matrix, ini_eq_rhs

def get_objective(m):
    ini_q_matrix = m.getQ().toarray()
    ini_p_vec = []
    variables = m.getVars()
    for var in variables:
        ini_p_vec.append(var.obj)
    ini_p_vec = np.array(ini_p_vec)

    return ini_q_matrix, ini_p_vec

def get_bound(m):
    variables = m.getVars()
    ini_lb = []
    ini_ub = []
    for var in variables:
        ini_lb.append(var.lb)
        ini_ub.append(var.ub)
    ini_lb = np.array(ini_lb)
    ini_ub = np.array(ini_ub)

    return ini_lb, ini_ub

## Get Q, p, A, G, h

In [6]:
Main.eval("""
pm = instantiate_model(network_data_orig, DCPPowerModel, PowerModels.build_opf);
JuMP.write_to_file(pm.model, "dcopf_model.lp")
""")

In [7]:
m = gp.read('/home/jxxiong/A-xjx/DC3/datasets/dcopf/dcopf_model.lp')
G, h, A, _ = get_constraints(m)
Q, p = get_objective(m)
Lb, Ub = get_bound(m)

Read LP format model from file /home/jxxiong/A-xjx/DC3/datasets/dcopf/dcopf_model.lp
Reading time = 0.00 seconds
obj: 936 rows, 484 columns, 2244 nonzeros


In [8]:
print(G.shape, h.shape, A.shape, Q.shape, p.shape, Lb.shape, Ub.shape)

(490, 484) (490,) (446, 484) (484, 484) (484,) (484,) (484,)


## Generate eq_rhs (X)

In [10]:
num_samples = 10000
objective_list = []
solve_time_list = []
X = []

In [11]:
while len(X) < num_samples:
    Main.eval("""
        network_data = deepcopy(network_data_orig)
        for (load_id, load) in network_data["load"]
            perturbation_factor = 0.8 + (rand() * 0.4);
            load["pd"] *= perturbation_factor;
            load["qd"] *= perturbation_factor;
        end

        pm = instantiate_model(network_data, DCPPowerModel, PowerModels.build_opf);
        result = optimize_model!(pm, optimizer=optimizer);
        status = (result["termination_status"] == MOI.LOCALLY_SOLVED || result["termination_status"] == MOI.OPTIMAL);
        obj = result["objective"];
        time = result["solve_time"];
    """)

    if Main.status:
        objective_list.append(Main.obj)
        solve_time_list.append(Main.time)
        Main.eval("""JuMP.write_to_file(pm.model, "dcopf_model.lp")""")
        m = gp.read('/home/jxxiong/A-xjx/DC3/datasets/dcopf/dcopf_model.lp')
        _, _, _, x = get_constraints(m)
        X.append(x)
    
    if len(X) % 100 == 0 and len(X) != 0:
        # save the data A, X, G, h, Q, p, lb, ub
        data = {'Q':Q, 'p':p, 'A':A, 'X':X, 'G':G, 'h':h, 'Lb':Lb, 'Ub':Ub, 'Y':[], 'objective':objective_list, 'solve_time':solve_time_list}
        with open(f"dcopf_data", 'wb') as f:
            pickle.dump(data, f)

with open(f"dcopf_data", 'wb') as f:
    pickle.dump(data, f)



******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

Read LP format model from file /home/jxxiong/A-xjx/DC3/datasets/dcopf/dcopf_model.lp
Reading time = 0.00 seconds
obj: 936 rows, 484 columns, 2244 nonzeros
Read LP format model from file /home/jxxiong/A-xjx/DC3/datasets/dcopf/dcopf_model.lp
Reading time = 0.00 seconds
obj: 936 rows, 484 columns, 2244 nonzeros
Read LP format model from file /home/jxxiong/A-xjx/DC3/datasets/dcopf/dcopf_model.lp
Reading time = 0.00 seconds
obj: 936 rows, 484 columns, 2244 nonzeros
Read LP format model from file /home/jxxiong/A-xjx/DC3/datasets/dcopf/dcopf_model.lp
Reading time = 0.00 seconds
obj: 936 rows, 484 columns, 2244 no

## Load Data

In [25]:
with open("dcopf_data", 'rb') as f:
    dataset = pickle.load(f)

In [18]:
nonzero_indices = np.nonzero(Q)
print(nonzero_indices)


(array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 13, 14, 15, 16, 17, 18,
       19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32]), array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 13, 14, 15, 16, 17, 18,
       19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32]))


In [20]:
np.nonzero(p)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32]),)

In [11]:
lb_index = np.where(Lb != -np.inf)[0]
ub_index = np.where(Ub != np.inf)[0]

In [16]:
np.where(Lb == Ub)[0]

array([240, 349, 382, 403, 434, 483])

In [14]:
intersection = np.intersect1d(lb_index, ub_index)
print(intersection)

[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32 233 234 235 236
 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254
 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272
 273 274 275 276 277 278 279 280 281 282 283 284 285 286 287 288 289 290
 291 292 293 294 295 296 297 298 299 300 301 302 303 304 305 306 307 308
 309 310 311 312 313 314 315 316 317 318 319 320 321 322 323 324 325 326
 327 328 329 330 331 332 333 334 335 336 337 338 339 340 341 342 343 344
 345 346 347 348 349 350 351 352 353 354 355 356 357 358 359 360 361 362
 363 364 365 366 367 368 369 370 371 372 373 374 375 376 377 378 379 380
 381 382 383 384 385 386 387 388 389 390 391 392 393 394 395 396 397 398
 399 400 401 402 403 404 405 406 407 408 409 410 411 412 413 414 415 416
 417 418 419 420 421 422 423 424 425 426 427 428 429 430 431 432 433 434
 435 436 437 438 439 440 441 442 443 444 445 446 44

In [21]:
Lb[240]

1.335

In [22]:
Ub[240]

1.335